<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: calwebb_image3, NIRCam imaging

<span style="color:red"> **Instruments Affected**</span>: e.g., NIRCam 

### Table of Contents

<div style="text-align: left"> 
    
<br> [Introduction\*](#intro)
<br> [JWST CalWG Algorithm\*](#algorithm)
<br> [Defining Terms](#terms)
<br> [Test Description\*](#description)
<br> [Data Description\*](#data_descr)
<br> [Imports\*](#imports)
<br> [Convenience Functions](#convenience_functions)
<br> [Loading the Data\*](#data_load)
<br> [calwebb_image3 - Ensemble calibrations](#image3) 
<br> [Run the entire pipeline](#image3_at_once)
<br> [Run the individual pipeline steps](#image3_step_by_step)
   <br> [The `WCS Refinement` step](#tweakreg)
   <br> [The `Sky Matching` step](#skymatch)
   <br> [The `Outlier Detection` step](#outlier_detection)
   <br> [The `Resample` step](#resample)
   <br> [The `Source Catalog` step](#source_catalog)
<br> [About This Notebook\*](#about)
<br>    

</div>

<a id='intro'></a>
# Introduction

This notebook covers part 3 of the imaging mode data calibration module. In this notebook we'll check Stage 3 of the JWST calibration pipeline for imaging data, also known as *calwebb\_image3*. 

The [Stage 3 pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_image3.html) takes one or more calibrated slope images (`*_cal.fits` files) and combines them into a final mosaic image. It then creates a source catalog from this mosaic. Several steps are performed in order to prepare the data for the mosaic creation. These steps largely mirror what is done by [DrizzlePac](https://www.stsci.edu/scientific-community/software/drizzlepac.html) software when working with HST data. 

First, using common sources found across the input images, the WCS of each image is refined. Background levels are then matched across the inputs. Spurious sources (e.g. cosmic rays that were not flagged in the `jump` step during Stage 1 processing) are removed by comparing each individual input image to a median image. The indivudal images are combined into a single mosaic image. A source catalog is created based on the mosaic image. And finally, the individual exposures are updated using the information from the preceding steps. New versions of the individual calibrated slope images (`*_cal.fits` files) are produced that contain matched backgrounds, flagged spurious sources, and improved WCS objects. Also, updated [resampled](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) images (`_i2d.fits` files) are created which all contain the final undistorted sky projection that is present in the mosaic image.

There are three final outputs. The first is updated copies of the input files. These updated files contain a consistent WCS, such that they overlap correctly. The second output is a final mosaic image created by drizzling the input images onto a distortion-free grid. And the final output is a source catalog wth basic photometry, created from the final mosaic image.

To check how the steps of the pipeline change the input data, we will use several NIRCam simulated data files and run them through the pipeline, examining the results at several places along the way.

All JWST imaging mode data, regardless of instrument, are processed through the *calwebb\_image3* pipeline. The steps and the order in which they are performed is the same for all data.

Pipeline description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image3.html

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst

[Top of Page](#title_ID)

<a id="algorithm"></a>
# JWST CalWG Algorithm

Multiple exposures from the direct imaging modes are combined into a combined image (e.g., mosaic).
The current status of the algorithms for this pipeline stage is summarized in the link below. Links are provided to individual pages where the details of the algorithms are given along with notes on why those algorithms were picked.

The algorithms for each step in each pipeline stage are split into baseline" and "enhanced" versions (formerly known as "vanilla" and "optimal", respectively).  See Baseline and Enhanced Algorithms for more details.
Input/Outputs of this stage refer to the main data products for the pipeline process. The full list of archive products for this (and all stages of the pipeline) is tabulated in Archive Products.

[JWST CalWG algorithms for calwebb_image3](https://outerspace.stsci.edu/display/JWSTCC/CALWEBB_IMAGE3)

[Top of Page](#title_ID)

<a id="terms"></a>
# Defining Terms


**JWST**: James Webb Space Telescope

**NIR**: Near Infrared


[Top of Page](#title_ID)

<a id="description"></a>
# Test Description

This test is performed by running simulated data through the full pipeline and performing a visual inspection of the outputs. Next, the notebook does quick checks after each step in the calwebb_image3 pipeline, based on the algorithms defined. 


[Top of Page](#title_ID)

<a id="data_descr"></a>
# Data Description

For this module, we will use an association of calibrated NIRCam simulated imaging exposures generated with Mirage.


[Top of Page](#title_ID)

<a id="tempdir"></a>
# Set up Temporary Directory
The following cell sets up a temporary directory (using python's `tempfile.TemporaryDirectory()`), and changes the script's active directory into that directory (using python's `os.chdir()`). This is so that, when the notebook is run through, it will download files to (and create output files in) the temporary directory rather than in the notebook's directory. This makes cleanup significantly easier (since all output files are deleted when the notebook is shut down), and also means that different notebooks in the same directory won't interfere with each other when run by the automated webpage generation process.

If you want the notebook to generate output in the notebook's directory, simply don't run this cell.

If you have a file (or files) that are kept in the notebook's directory, and that the notebook needs to use while running, you can copy that file into the directory (the code to do so is present below, but commented out).

[Top of Page](#title_ID)

In [1]:
#****
#
# Set this variable to False to not use the temporary directory
#
#****
use_tempdir = True

# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
import shutil

if use_tempdir:
    data_dir = TemporaryDirectory()

    # If you have files that are in the notebook's directory, but that the notebook will need to use while
    # running, copy them into the temporary directory here.
    #
    # files = ['name_of_file']
    # for file_name in files:
    #     shutil.copy(file_name, os.path.join(data_dir.name, file_name))

    # Save original directory
    orig_dir = os.getcwd()

    # Move to new directory
    os.chdir(data_dir.name)

# For info, print out where the script is running
print("Running in {}".format(os.getcwd()))

Running in /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpf_50kwdz


## If Desired, set up CRDS to use a local cache

By default, the notebook template environment sets up its CRDS cache (the "CRDS_PATH" environment variable) in /grp/crds/cache. However, if the notebook is running on a local machine without a fast and reliable connection to central storage, it makes more sense to put the CRDS cache locally. Currently, the cell below offers several options, and will check the supplied boolean variables one at a time until one matches.

* if `use_local_crds_cache` is False, then the CRDS cache will be kept in /grp/crds/cache
* if `use_local_crds_cache` is True, the CRDS cache will be kept locally
  * if `crds_cache_tempdir` is True, the CRDS cache will be kept in the temporary directory
  * if `crds_cache_notebook_dir` is True, the CRDS cache will be kept in the same directory as the notebook.
  * if `crds_cache_home` is True, the CRDS cache will be kept in $HOME/crds/cache
  * if `crds_cache_custom_dir` is True, the CRDS cache will be kept in whatever is stored in the 
    `crds_cache_dir_name` variable.

If the above cell (creating a temporary directory) is not run, then setting `crds_cache_tempdir` to True will store the CRDS cache in the notebook's directory (the same as setting `crds_cache_notebook_dir` to True).

In [2]:
import os

# Choose CRDS cache location
use_local_crds_cache = False
crds_cache_tempdir = False
crds_cache_notebook_dir = False
crds_cache_home = False
crds_cache_custom_dir = False
crds_cache_dir_name = ""

if use_local_crds_cache:
    if crds_cache_tempdir:
        os.environ['CRDS_PATH'] = os.path.join(os.getcwd(), "crds")
    elif crds_cache_notebook_dir:
        try:
            os.environ['CRDS_PATH'] = os.path.join(orig_dir, "crds")
        except Exception as e:
            os.environ['CRDS_PATH'] = os.path.join(os.getcwd(), "crds")
    elif crds_cache_home:
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif crds_cache_custom_dir:
        os.environ['CRDS_PATH'] = crds_cache_dir_name

<a id='Imports_ID'></a>
# Imports

Import packages necessary for this notebook

In [3]:
# Module with functions to get information about objects:
from glob import glob
import os
import shutil

# Numpy library:
import numpy as np

# To read association file
import json

# To download data
import requests

# To examine parameter reference files
import asdf

# Astropy tools:
from astropy.io import ascii, fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch

Set up matplotlib for plotting

In [4]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
# %matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

Import JWST pipeline-related modules

In [5]:
# The entire calwebb_image3 pipeline
from jwst.pipeline import calwebb_image3

# Individual steps that make up calwebb_image3
from jwst.tweakreg import TweakRegStep
from jwst.skymatch import SkyMatchStep
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.source_catalog import SourceCatalogStep
from jwst import datamodels
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

Check which version of the pipeline we are running:

In [6]:
import jwst
print(jwst.__version__)

1.3.1


<a id='convenience_functions'></a>
# Define convenience functions and parameters

Here we define some functions that we will use repeatedly throughout the notebook.

In [7]:
# Files created in this notebook will be saved
# in a subdirectory of the base directory called `Stage3`
output_dir = './'

In [8]:
def find_bad_pix_types(dq_value):
    """Given an integer representation of a series of bad pixel flags,
    identify which types of bad pixels the flags indicate.
    
    Parameters
    ----------
    dq_value : uint16
        Value associated with a set of bad pixel flags
        
    Returns
    -------
    bad_nums : list
        List of integers representing the bad pixel types
        
    bad_types : list
        List of bad pixel type names corresponding to bad_nums
    """
    # Change integer into a byte array
    bitarr = np.binary_repr(dq_value)
    
    # Find the bad pixel type associated with each bit where
    # the flag is set
    bad_nums = []
    bad_types = []
    for i, elem in enumerate(bitarr[::-1]):
        if elem == str(1):
            badval = 2**i
            bad_nums.append(badval)
            key = next(key for key, value in datamodels.dqflags.pixel.items() if value == badval)
            bad_types.append(key)
    return bad_nums, bad_types

In [9]:
def overlay_catalog(data_2d, catalog, flux_limit=0, vmin=0, vmax=10,
                    title=None, units='MJy/str'):
    """Function to generate a 2D image of the data, 
    with sources overlaid.
    
    data_2d : numpy.ndarray
        2D image to be displayed
        
    catalog : astropy.table.Table
        Table of sources
    
    flux_limit : float
        Minimum signal threshold to overplot sources from catalog.
        Sources below this limit will not be shown on the image.
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    title : str
        String to use for the plot title
                
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    for row in catalog:
        if row['aper_total_flux'].value > flux_limit:
            plt.plot(row['xcentroid'], row['ycentroid'], marker='o',
                     markersize='3', color='red')

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    fig.colorbar(im, label=units)
    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)

In [10]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None,
               scale='log', units='MJy/str'):
    """Function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel.
    
    data_2d : numpy.ndarray
        2D image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
        
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear'
        
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

[Top of Page](#title_ID)

<a id="data_load"></a>
# Loading the Data

For this module, we will use calbrated rate files from a NIRCam simulated imaging exposure that is stored in Box. Let's download these files, as well as an association file and some parameter reference files.

In [11]:
from astropy.utils.data import download_file
from pathlib import Path
from shutil import move
from os.path import splitext

def get_box_files(file_list):
    for box_url,file_name in file_list:
        if 'https' not in box_url:
            box_url = 'https://stsci.box.com/shared/static/' + box_url
        downloaded_file = download_file(box_url)
        if Path(file_name).suffix == '':
            ext = splitext(box_url)[1]
            file_name += ext
        move(downloaded_file, file_name)
        
file_urls = ['https://stsci.box.com/shared/static/p2wlvndw25dk7xwk1tasqevmhkg6p55e.fits',
             'https://stsci.box.com/shared/static/cmhc7kkf5z6373d2vwg7916lrhe5ia7u.fits',
             'https://stsci.box.com/shared/static/sb18cpfqjbw1i09gvw0cpqkj6ymdn899.fits',
             'https://stsci.box.com/shared/static/7d00b9isvss7njhcwmd8uiq8c7s4d845.json',
             'https://stsci.box.com/shared/static/ja0gkd8c0x8p8konhr84wkuhwnpkqf4s.asdf',
             'https://stsci.box.com/shared/static/yahdw55fotwrh7i6hhcxksj97qkf7j4r.asdf',
              ]

file_names = ['jw98765001001_01101_00001_nrcb5_cal.fits',
              'jw98765001001_01101_00002_nrcb5_cal.fits',
              'jw98765001001_01101_00003_nrcb5_cal.fits',
              'level3_lw_asn.json',
              'jwst_nircam_pars-tweakregstep_0006.asdf',
              'nircam_pars-sourcecatalogstep_f444w_clear.asdf'
              ]  

box_download_list = [(url,name) for url,name in zip(file_urls,file_names)]

In [12]:
get_box_files(box_download_list)

In [13]:
asn_file = os.path.join(output_dir, 'level3_lw_asn.json')

In [14]:
# Open the association file and load into a json object
with open(asn_file) as f_obj:
    asn_data = json.load(f_obj)

In [15]:
asn_data

{'asn_type': 'None',
 'asn_rule': 'DMS_Level3_Base',
 'version_id': None,
 'code_version': '0.17.1',
 'degraded_status': 'No known degraded exposures in association.',
 'program': 'noprogram',
 'constraints': 'No constraints',
 'asn_id': 'a3001',
 'target': 'none',
 'asn_pool': 'none',
 'products': [{'name': 'l3_lw_results',
   'members': [{'expname': 'jw98765001001_01101_00001_nrcb5_cal.fits',
     'exptype': 'science'},
    {'expname': 'jw98765001001_01101_00002_nrcb5_cal.fits',
     'exptype': 'science'},
    {'expname': 'jw98765001001_01101_00003_nrcb5_cal.fits',
     'exptype': 'science'}]}]}

Here we see that the association file begins with a few lines of data that give high-level information about the association. The most important entry here is the `asn_rule` field. Association files have different formats for the different stages of the pipeline. You should be sure that the `asn_rule` matches the pipeline that you will be running. In this case we'll be running the Stage 3 pipeline, and we see that the `asn_rule` mentions "Level3", which is what we want.

Beneath these lines, we see the `products` field. This field contains a list of dictionaries that specify the files that belong to this association, and the types of those files. When the Stage 3 pipeline is run on this association file, all files listed here will be run through the calibration steps.

In [16]:
tweak_files = ['level3_lw_asn_0_tweakregstep.fits',
               'level3_lw_asn_1_tweakregstep.fits',
               'level3_lw_asn_2_tweakregstep.fits']
tweak_product = 'manual_asn_file'

In [17]:
tweakreg_asn = asn_from_list.asn_from_list(tweak_files, rule=DMS_Level3_Base, product_name=tweak_product)

Here is our new association, containing the three files created by the `tweakreg` step.

In [18]:
tweakreg_asn

{
    "asn_type": "None",
    "asn_rule": "DMS_Level3_Base",
    "version_id": null,
    "code_version": "1.3.1",
    "degraded_status": "No known degraded exposures in association.",
    "program": "noprogram",
    "constraints": "No constraints",
    "asn_id": "a3001",
    "target": "none",
    "asn_pool": "none",
    "products": [
        {
            "name": "manual_asn_file",
            "members": [
                {
                    "expname": "level3_lw_asn_0_tweakregstep.fits",
                    "exptype": "science"
                },
                {
                    "expname": "level3_lw_asn_1_tweakregstep.fits",
                    "exptype": "science"
                },
                {
                    "expname": "level3_lw_asn_2_tweakregstep.fits",
                    "exptype": "science"
                }
            ]
        }
    ]
}

Now save the new association to a json file.

In [19]:
output_test = 'manual_tweakreg_asn.json'
with open(output_test, 'w') as outfile:
    name, serialized = tweakreg_asn.dump(format='json')
    outfile.write(serialized)

[Top of Page](#title_ID)

---
<a id='image3'></a>
# The calwebb_image3 pipeline: Ensemble processing

In the sections below, we will run the Stage 3 pipeline using an association file containing several NIRCam exposures. We will first call the entire *calwebb_image3* pipeline itself. The pipeline is a wrapper which will string together all of the appropriate steps in the proper order.

After running the entire pipeline, we will go back to the original calibrated slope images and manually run them through each of the steps that comprise the Stage 3 pipeline. For each step we will check in more detail what is going on and examine how the exposure files have changed.

See [Figure 1](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/algorithm-documentation/stages-of-processing/calwebb_image3) on the calwebb_image3 algorithm page for a map of the steps are performed on the input data.

<a id='image3_at_once'></a>
# Run the entire `calwebb_image3` pipeline

In this section we show how to run the entire calwebb_image3 pipeline with a single call. 

We set parameter values for some of the individual steps, save some outputs, etc, and then call the pipeline.

In [20]:
# Create an instance of the pipeline class
image3 = calwebb_image3.Image3Pipeline()

# Set some parameters that pertain to the
# entire pipeline
image3.output_dir = output_dir
image3.save_results = True
image3.tweakreg.save_results = True
image3.skymatch.save_results = True
image3.outlier_detection.save_results = True
image3.resample.save_results = True
image3.source_catalog.save_results = True

# Set some parameters that pertain to some of
# the individual steps
image3.tweakreg.snr_threshold = 10.0  # 5.0 is the default
image3.tweakreg.kernel_fwhm = 2.302  # 2.5 is the default
image3.tweakreg.brightest = 20  # 100 is the default
image3.tweakreg.align_to_gaia = True
image3.tweakreg.save_gaia_catalog = True
image3.source_catalog.kernel_fwhm = 2.302  # pixels
image3.source_catalog.snr_threshold = 10.

# Call the run() method
image3.run(asn_file)

2021-10-01 18:24:30,033 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.


2021-10-01 18:24:30,035 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.


2021-10-01 18:24:30,037 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.


2021-10-01 18:24:30,039 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.


2021-10-01 18:24:30,041 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.


...

2021-10-01 18:27:17,974 - stpipe.Image3Pipeline.source_catalog - INFO - Subarray: FULL


2021-10-01 18:27:18,069 - stpipe.Image3Pipeline.source_catalog - INFO - AB to Vega magnitude offset 3.24181


2021-10-01 18:27:19,632 - stpipe.Image3Pipeline.source_catalog - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/jwst/source_catalog/source_catalog.py:422: AstropyDeprecationWarning: "filter_kernel" was deprecated in version 1.2 and will be removed in a future version. Use argument "kernel" instead.
  segm = detect_sources(data, threshold, npixels, filter_kernel=kernel,



2021-10-01 18:27:20,199 - stpipe.Image3Pipeline.source_catalog - INFO - Detected 340 sources


ValueError: too many values to unpack (expected 2)

### Examine the outputs

Get the input filenames from the association file

In [ ]:
input_files = [item['expname'] for item in asn_data['products'][0]['members']]       

In [ ]:
input_files

Define the names of the other output files.

In [ ]:
mosaic_file = os.path.join(output_dir, 'l3_lw_results_i2d.fits')
source_cat_file = os.path.join(output_dir, 'l3_lw_results_cat.ecsv')
segmentation_map_file = os.path.join(output_dir, 'l3_lw_results_segm.fits')
cr_flagged_files = [item.replace('cal.fits', 'crf.fits') for item in input_files]

Read in the final mosaic image and display

In [ ]:
#with datamodels.open(mosaic_file) as mosaic:
mosaic = datamodels.open(mosaic_file)
print(mosaic.info())

In [ ]:
show_image(mosaic.data, vmin=0, vmax=5)

Let's look at the segmentation map that was created by the `source_catalog` step. This shows which pixels are associated with the identified sources.

In [ ]:
seg_map = fits.getdata(segmentation_map_file)

In [ ]:
show_image(seg_map, vmin=0, vmax=5, scale='linear')

And now examine the actual source catalog. For each source, the catalog lists the location, along with flux and AB/Vega magnitude values in three different apertures, as well as calculated values for an infinite aperture. Within the documentation, you can see the [full list of column definitions](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html#source-catalog-table).

In [ ]:
source_cat = ascii.read(source_cat_file)

In [ ]:
source_cat

Finally, let's overlay the source catalog on top of the mosaic image. In order to cut down on the number of spurious detections, we only show sources above a minimum flux limit. Another way to cut down on the number of spurious detections would be to change some of the `source_catalog` parameter values when calling the pipeline above.

In [ ]:
overlay_catalog(mosaic.data, source_cat, flux_limit=5e-7, vmin=0, vmax=10,
                title='Final mosaic with source catalog')

[Top of Page](#title_ID)

<a id='image3_step_by_step'></a>
# Run the individual pipeline steps

In the sections below we run the steps contained within calwebb_image3 one at a time, in order to check the outputs for each step.

<a id='tweakreg'></a>
### The `WCS refinement` step

#### Summary

This step, called the `tweakreg` step, mimics the behavior of the tweakreg step of Astrodrizzle. Given a series of images, it identifies point sources that are common to two or more images, and uses those sources' locations to correct the WCS of the input images. The tweaks are such that when the images are later combined into a final mosiac image, the WCS of the input images will align on the sky. 

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/tweakreg/README.html) of the step.

#### Arguments

There are many [optional input arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/tweakreg/README.html#step-arguments).

#### Reference files used

This step does not use any reference files.

#### Parameter reference files

There are filter-dependent parameter reference files in CRDS for this step. 

#### Run the step

Since we ran the step above and saved the outputs, we'll skip this part and just load the outputs. 

Let's look at what parameters are available to be set.

In [ ]:
print(TweakRegStep.spec)

In [ ]:
# # Create instance, set parameters, and run
# tweakreg = TweakRegStep()
# tweakreg.kernel_fwhm = 2.302   # Gaussian FWHM in pixels
# tweakreg.snr_threshold = 10.0  # SNR threshold above background
# tweakreg.brightest = 100       # Number of brightest objects to keep
# tweakreg.save_results = True
# tweak = tweakreg.run(asn_file)

The step saves the results in new fits files with updated WCS information. Let's look at the difference in the WCS before and after the step by loading the WCS objects, and calculating the RA, Dec at one pixel.

In [ ]:
cal_file = 'jw98765001001_01101_00003_nrcb5_cal.fits'
tweak_file = 'jw98765001001_01101_00003_nrcb5_tweakreg.fits'

In [ ]:
# Open the files using datamodels
#with datamodels.open(cal_file) as cal_data:
cal_data = datamodels.open(cal_file)
print(type(cal_data))
#with datamodels.open(tweak_file) as tweak_data:
tweak_data = datamodels.open(tweak_file)
print(type(tweak_data))

In [ ]:
cal_detector_to_world = cal_data.meta.wcs.get_transform('detector', 'world')
run_detector_to_world = tweak_data.meta.wcs.get_transform('detector', 'world')

Let's look at RA, Dec in the center of the detector.

In [ ]:
x, y = (1024, 1024)

In [ ]:
print(cal_detector_to_world(x, y))
print(run_detector_to_world(x, y))

In [ ]:
# What is the shift in the WCS before/after this step?
delta_ra = run_detector_to_world(x, y)[0] - cal_detector_to_world(x, y)[0]
delta_dec = run_detector_to_world(x, y)[1] - cal_detector_to_world(x, y)[1]
print('Shift in RA, Dec is ({:.4f}, {:.4f}) arcsec'.format(delta_ra * 3600., delta_dec * 3600.))
print('This is ({:.3f}, {:.3f}) pixels.'.format(delta_ra * 3600. / .062, delta_dec * 3600. / 0.062))

This extremely small shift is expected, since the simulated data used in this exercise has no jitter or offsets added to it.

Just for fun, let's look at the contents of the WCS object in one of the files saved by the tweakreg step.

In [ ]:
tweak_data.meta.wcs

[Top of Page](#title_ID)

<a id='skymatch'></a>
## The `Sky matching` step

#### Summary

This step calculates sky values in overlapping regions of the input images. Sky values can be computed for each image separately or in a way that matches the sky levels amongst the collection of images so as to minimize their differences.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/skymatch/README.html) of the step. Note that there are several possible methods for calculating the sky values. 

#### Arguments

There are [several optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/skymatch/README.html#step-arguments) for this step.

#### Reference files used

This step does not use any reference files.

#### Parameter reference files
There are currently no parameter reference files for this step

#### Run the step

Since we ran the step above and saved the outputs, we'll skip this part and just load the outputs. 

        The easiest way to call this step is to supply the output object from the tweakreg call above. The other option would be to supply an association file. However, since the tweakreg step saved modified files, we would have to create a new association file that contains these new files, and then supply that association file in the call to skymatch. This pattern is the same for all of the steps in calwebb_image3, since all steps now expect association files for inputs, rather than individual files. 

        To create a new association file, follow the steps outlined in the [Creating your own association files](#diy_association) section above. Keep in mind that you could also simply make a copy of an existing association file and update the member filenames.

Let's also see what parameters are available to be set.

In [ ]:
print(SkyMatchStep.spec)

In [ ]:
# skymatch = SkyMatchStep()
# skymatch.skymethod = 'global+match' # this is the default. Set here as an example
# skymatch.save_results = True
# sky = skymatch.run(tweak)

If you request to save the output from this step, two new header keywords are added to the primary header of the outputs. In each file, the `BKGLEVEL` keyword lists the computed background level, and the `BKGSUB` keyword says whether or not the background has been subtracted from the data.

In this case, we kept the default behavior of not subtracting the background from the data, as shown below.

In [ ]:
# sky_file = 'step_SkyMatchStep_2_skymatchstep.fits'
sky_file = 'l3_lw_results_2_skymatch.fits'

In [ ]:
sky_header = fits.getheader(sky_file, 0)
print('Computed background level: {}'.format(sky_header['BKGLEVEL']))
print('Background subtracted: {}'.format(sky_header['BKGSUB']))

Here we see that the calculated background value is not subtracted from the input data:

In [ ]:
#with datamodels.open(sky_file) as sky_data:
sky_data = datamodels.open(sky_file)
print(sky_data.info())

In [ ]:
np.min(sky_data.data - cal_data.data), np.max(sky_data.data - cal_data.data)

In [ ]:
sky_data.close()

[Top of Page](#title_ID)

<a id='outlier_detection'></a>
## The `Outlier Detection` step

#### Summary

This step uses the collection of input files to identify and flag any cosmic rays or other transient image artifacts that were not flagged by the jump step in calwebb_detector1. While the jump step looked for large pixel-based deviations in the signal from group-to-group within an integration, the outlier detection step looks for large sky-based exposure-to-exposure devations in the signal. If a given location on the sky shows no signal above the noise in 4 out of 5 exposures, but a bright source in the remaining exposure, the outlier detction step will flag in the DQ map the pixels containing the source in the fifth exposure. These pixels will be ignored when the exposures are later combined into a final mosaic image.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/main.html) of the step.

#### Arguments

There are [numerous optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/arguments.html) for this step, including several that apply to the resample step, which this step makes use of.


#### Reference files used

This step does not use any reference files.

#### Run the step

Since we ran the step above and saved the outputs, we'll skip this part and just load the outputs. 

List the available parameters:

In [ ]:
print(OutlierDetectionStep.spec)

In [ ]:
# # Run the step
# outlier_detection = OutlierDetectionStep()
# outlier_detection.save_results = True
# outlier = outlier_detection.run(sky)

In [ ]:
# outlier_file = 'step_SkyMatchStep_2_a3001_outlierdetectionstep.fits'
outlier_file = 'l3_lw_results_2_a3001_crf.fits'

In [ ]:
# Open using datamodels
#with datamodels.open(outlier_file) as outlier_data:
outlier_data = datamodels.open(outlier_file)
print(outlier_data.info())

In [ ]:
# Get a list of pixels where DQ flags were changed
new_flags = np.where(cal_data.dq != outlier_data.dq)

In [ ]:
print("Found {} pixels with updated DQ flag values.".format(len(new_flags[0])))

Let's have a look at the DQ value for one of these pixels before and after the outlier detection step has run, in order to see what has changed.

In [ ]:
index = 12515
y = new_flags[0][index]
x = new_flags[1][index]
print('Pixel x, y = ({}, {})'.format(x, y))
print('Before: {}'.format(cal_data.dq[y, x]))
print('After: {}'.format(outlier_data.dq[y, x]))

Below we see that the 'OUTLIER' and the 'DO_NOT_USE' flags are new. 

In [ ]:
find_bad_pix_types(cal_data.dq[y, x])

In [ ]:
find_bad_pix_types(outlier_data.dq[y, x])

The image data itself remains unchanged.

In [ ]:
print(cal_data.data[y, x], outlier_data.data[y, x])

When the Resample step is run next in order to create the final mosaic image, all pixels flagged as DO_NOT_USE will be ignored.

[Top of Page](#title_ID)

<a id='resample'> </a>
## The `Resample` step

#### Summary

We initially saw this step in calwebb_image2, where it was used to resample individual images onto a distortion-free pixel grid. This time, the Resample step works on the set of input images, which now all have a consistent WCS, thanks to the tweakreg step. The input images are combined into a final mosaic as they are resampled onto a distortion-free grid. The output of this step is the final image output of the Stage 3 pipeline. 

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) of the step.

#### Arguments

There is a list of [optional Astrodrizzle-style](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/arguments.html) input parameters that can be used to customize the resampling process.

#### Reference files used

This step uses the [`DRIZPARS`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/reference_files.html) reference file. This file contains Astrodrizzle-style keywords that can be used to control the details of the resampling.

#### Run the step

Since we ran the step above and saved the outputs, we'll skip this part and just load the outputs. 

List the available parameters and their default values:

In [ ]:
print(ResampleStep.spec)

In [ ]:
# # Run the step
# resample = ResampleStep()
# resample.save_results = True
# resamp = resample.run(outlier)

In [ ]:
# resamp_file = 'step_ResampleStep_resamplestep.fits'
resamp_file = 'l3_lw_results_i2d.fits'

#with datamodels.open(resamp_file) as resamp:
resamp = datamodels.open(resamp_file)
print(resamp.info())

In [ ]:
show_image(resamp.data, 0, 10)

In [ ]:
resamp.data.shape

[Top of Page](#title_ID)

<a id='source_catalog'> </a>
## The `Source Catalog` step

#### Summary

This step creates a catalog of source photometry and morphology information. Sources are identified using [Photutils' image segmentation](https://photutils.readthedocs.io/en/latest/segmentation.html) method. The output is an ASCII file containing a table of source locations and aperture photometry results.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html) of the step.

#### Arguments

There is a list of [optional input parameters](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/arguments.html) that can be used to customize the resampling process.

#### Reference files used

This step uses the [`APCORR` and `ABVEGAOFFSET`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/reference_files.html) reference files. The `APCORR` reference file contains the factors necessary to correct aperture photometry results to the equivalent of an infinite aperure. The `ABVEGAOFFSET` reference file contains data necessary for converting from AB to Vega magnitudes.

<a id='srccat_run'></a>
#### Run the step with the run() method

Since we ran the step above and saved the outputs, we'll skip this part and just load the outputs. 

List the available parameters and their default values:

In [ ]:
print(SourceCatalogStep.spec)

In [ ]:
# Run the step after setting some parameters
srccat = SourceCatalogStep()
srccat.save_results = True
srccat.kernel_fwhm = 2.302  # pixels
srccat.snr_threshold = 10.

source_cat = srccat.run(resamp_file)

#### Examine the results

In [ ]:
source_cat

In [ ]:
overlay_catalog(resamp.data, source_cat, flux_limit=0, vmin=0, vmax=10,
                title='Final mosaic with source catalog')

[Top of Page](#title_ID)

<a id="about_ID"></a>
## About this Notebook
**Original Author:** Bryan Hilbert, updated by Alicia Canipe, NIRCam
<br>**Updated On:** 07/28/2021

[Top of Notebook](#title_ID)